Generate stl mesh from the binary masks

In [10]:
import meshlib.mrmeshpy as mr
import meshlib.mrmeshnumpy as mrn
import os
import SimpleITK as sitk

path = "/home/paulagmtz/TFM_PAULA_24/DATA_TFM/BINARY_MASKS/"
im_list = os.listdir(path)
for i in range(len(im_list)):
    inputData = path + im_list[i]
    name = os.path.splitext(im_list[i])[0]
    im = sitk.ReadImage(inputData, sitk.sitkFloat32)
    im_arr = sitk.GetArrayFromImage(im)
    #convert 3D array to SimpleVolume data
    simpleVolume = mrn.simpleVolumeFrom3Darray(im_arr)

    #convert SimpleVolume to FloatGrid data
    floatGrid = mr.simpleVolumeToDenseGrid(simpleVolume )

    #make mesh by iso-value = 0.5 and voxel size = (0.1, 0.1, 0.1)
    my_mesh = mr.gridToMesh(floatGrid , mr.Vector3f(0.1, 0.1, 0.1), 0.5)
    # print(f"/home/paulagmtz/TFM_PAULA_24/DATA_TFM/stl/{str(im_list[i])}_mesh.stl")

    #save mesh
    mr.saveMesh(my_mesh,f"/home/paulagmtz/TFM_PAULA_24/DATA_TFM/stl/{name}_mesh.stl")

Tengo stl y quiero pasarlo a OFF para que los archivos que generen sean como los del modelnet40. en estos archivos tengo descrito:

        OFF
        num_vertices num_faces num_edges
        vertices
        faces

Ejemplo:


        OFF
        8 6 12
        1.0 0.0 0.0
        0.0 1.0 0.0 
        ..
        4 0 1 5 4
        4 1 2 6 5
        ...

# Convert stl to off

In [13]:
from stl import mesh
import numpy as np

def convert_to_off(stl_filename, off_filename):
    stl_mesh = mesh.Mesh.from_file(stl_filename)

    # Obtener vértices y caras
    vertices = stl_mesh.vectors.reshape((-1, 3))
    faces = np.arange(len(vertices)).reshape((-1, 3))

    # Escribir en formato OFF
    with open(off_filename, 'w') as off_file:
        off_file.write('OFF\n')
        off_file.write(f'{len(vertices)} {len(faces)} 0\n')

        for vertex in vertices:
            off_file.write(f'{vertex[0]} {vertex[1]} {vertex[2]}\n')

        for face in faces:
            off_file.write(f'3 {face[0]} {face[1]} {face[2]}\n')


stl_folder = "/home/paulagmtz/TFM_PAULA_24/DATA_TFM/stl/"
off_folder = "/home/paulagmtz/TFM_PAULA_24/DATA_TFM/off/"
for i in os.listdir(stl_folder):
    
    stl_filename = stl_folder + i
    name = os.path.splitext(i)[0]
    
    off_filename = off_folder+name+".off"
    
    convert_to_off(stl_filename, off_filename)
